In [ ]:
# Standard Libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import math
import json
import os
import pickle 
%matplotlib inline

# Neural Network Libraries
import torch
import torch.nn as nn
import torchvision
from torch.autograd import Variable
from torchvision.utils import make_grid
import torch.nn.functional as F
from torchvision import datasets, transforms
import scipy.linalg as linalg
#for colab purposes only
from google.colab import drive
drive.mount('/content/gdrive')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
cuda


In [ ]:
nSamples = 5000

In [ ]:
#Courtesy https://github.com/vfcosta/coegan.git

def FID(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Numpy implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).
    Stable version by Dougal J. Sutherland.
    Params:
    -- mu1   : Numpy array containing the activations of a layer of the
               inception net (like returned by the function 'get_predictions')
               for generated samples.
    -- mu2   : The sample mean over activations, precalculated on an 
               representive data set.
    -- sigma1: The covariance matrix over activations for generated samples.
    -- sigma2: The covariance matrix over activations, precalculated on an 
               representive data set.
    Returns:
    --   : The Frechet Distance.
    """
    import time

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2
    start_time = time.time()

    # Product might be almost singular
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    # Numerical error might give slight imaginary component
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            # raise ValueError('Imaginary component {}'.format(m))
            # print('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1) +
            np.trace(sigma2) - 2 * tr_covmean)

In [ ]:
dataDim = 784 # 28x28
labelDim = 10 # 0-9 
noiseDim = 100 # Z dimension 

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 50)
        self.fc1_drop = nn.Dropout(0.2)
        self.fc2 = nn.Linear(50, 50)
        self.fc2_drop = nn.Dropout(0.2)
        self.fc3 = nn.Linear(50, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = self.fc1_drop(x)
        x = F.relu(self.fc2(x))
        x = self.fc2_drop(x)
        return F.log_softmax(self.fc3(x), dim=1)

encPath = '/content/gdrive/MyDrive/Colab Notebooks/GAN/mnistModel.pkl'
with open(encPath, 'rb') as f:
    classifier = pickle.load(f)

classifier.to(device)

Net(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc1_drop): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (fc2_drop): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_features=50, out_features=10, bias=True)
)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.labelEmbedding = nn.Embedding(labelDim, labelDim)
        
        self.model = nn.Sequential(
            nn.Linear(noiseDim + labelDim , 250),
            nn.LeakyReLU(leakyReLUNegSlope, inplace=True),
            nn.Linear(250, 500),
            nn.LeakyReLU(leakyReLUNegSlope, inplace=True),
            nn.Linear(500, 1000),
            nn.LeakyReLU(leakyReLUNegSlope, inplace=True),
            nn.Linear(1000, dataDim),
            nn.Tanh()
        )
    
    def forward(self, z, labels):
        z = z.view(z.size(0), noiseDim)
        c = self.labelEmbedding(labels)
        x = torch.cat([z, c], 1)
        out = self.model(x)
        return out.view(x.size(0),int(np.sqrt(dataDim)),int(np.sqrt(dataDim)))

def generateSamples(generator, nSamples, random=True):
    if not random:
        return generator(torch.randn(nSamples, noiseDim).to(device),
                     torch.LongTensor(np.arange(nSamples)%10).to(device))
    return generator(torch.randn(nSamples, noiseDim).to(device),
                     torch.LongTensor(np.random.randint(0, labelDim, nSamples)).to(device))



In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5], [0.5]) 
])



dataset = datasets.MNIST('./data', 
                          train=True, 
                          download=True,
                          transform=transform)

loader = torch.utils.data.DataLoader(dataset=dataset, 
                                      batch_size=nSamples, 
                                      shuffle=True)

x = (next(iter(loader))[0]).to(device)

activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook



output = classifier(x)
classifier.fc2.register_forward_hook(get_activation('fc2'))
realEncodings = activation['fc2']
muReal = realEncodings.mean(axis=0)
covReal = realEncodings.T.cov()

In [ ]:
def generatedEncondingMaker(genPath,nSamples=1000):
    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

    with open(genPath, 'rb') as f:
        generator = pickle.load(f)

    activation = {}
    classifier.fc2.register_forward_hook(get_activation('fc2'))
    x = generateSamples(generator, nSamples)
    output = classifier(x)
    generatedEncodings = activation['fc2']
    return generatedEncodings

In [ ]:
genPaths = {
    'log1':'/content/gdrive/MyDrive/Colab Notebooks/GAN/log1/generator',
    'log2':'/content/gdrive/MyDrive/Colab Notebooks/GAN/log2/generator',
    'log3':'/content/gdrive/MyDrive/Colab Notebooks/GAN/log3/generator',
    'log4':'/content/gdrive/MyDrive/Colab Notebooks/GAN/log4/generator',
    'log5':'/content/gdrive/MyDrive/Colab Notebooks/GAN/log5/generator',
    'log6':'/content/gdrive/MyDrive/Colab Notebooks/GAN/log6/generator',
    'log7':'/content/gdrive/MyDrive/Colab Notebooks/GAN/log7/generator',
    'log8':'/content/gdrive/MyDrive/Colab Notebooks/GAN/log8/generator.pkl',
    'log9':'/content/gdrive/MyDrive/Colab Notebooks/GAN/log9/generator.pkl',
    'log10':'/content/gdrive/MyDrive/Colab Notebooks/GAN/log10/generator.pkl'
}

for log,genPath in genPaths.items():
    generatedEncodings = generatedEncondingMaker(genPath,nSamples = nSamples)
    muGenerated = generatedEncodings.mean(axis=0)
    covGenerated = generatedEncodings.T.cov()
    fid = FID(muReal.to('cpu'), covReal.to('cpu'), muGenerated.to('cpu'), covGenerated.to('cpu'))
    print('FID for generator with logfile \"'+ log + '\" is ' + str(round(fid,4)))

FID for generator with logfile "log1" is 2.6929
FID for generator with logfile "log2" is 1.271
FID for generator with logfile "log3" is 1.7456
FID for generator with logfile "log4" is 3.6701
FID for generator with logfile "log5" is 167.5097
FID for generator with logfile "log6" is 1.7497
FID for generator with logfile "log7" is 2.0212
FID for generator with logfile "log8" is 0.9597
FID for generator with logfile "log9" is 0.6166
FID for generator with logfile "log10" is 1.4947


In [ ]:
print(fid)